In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [21]:
def similarity(s1, s2):
    from sentence_transformers import SentenceTransformer, util
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    s=[]
    s.append(s1)
    s.append(s2)
    sentence_embeddings = model.encode(s)
    # print(sentence_embeddings)

    sim=util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1])
    # print(sim)
    return sim

similarity('hey', 'hi')

tensor([[0.9607]])

In [24]:
import json
with open('esprompt_500.jsonl', 'r') as json_file1:
    pred = list(json_file1)
    pred_num=len(pred)
    print(pred_num)

with open('questions.json', 'r') as json_file2:
    questions = json.load(json_file2)
#     # qs_num=len(questions)
#     # print(qs_num)

with open('subset_answers.json', 'r') as json_file3:
    egoschema_ans = json.load(json_file3)
    gts=[]
    for gt in egoschema_ans:
        gts.append(gt)
    print(gts)
    gts_num=len(gts)
    print(gts_num)

ans1, ans2, ans3=0,0,0
i=0
# currentpred=0
for pred_list in pred:
    result = json.loads(pred_list)
    question_id=result['id']
    if question_id not in egoschema_ans:
        continue
    pred=result['pred']
    print("question_id: ", question_id)
    print("pred: ", pred)
    prompt=[]
    for ans in questions:
        if question_id==ans['q_uid']:
            prompt.append(ans['option 0'])
            prompt.append(ans['option 1'])
            prompt.append(ans['option 2'])
            prompt.append(ans['option 3'])
            prompt.append(ans['option 4'])
    print("prompt: ", prompt)
    sims=[]
    p_i=0
    for p in pred:
        # p_i+=1
        for q in prompt:
            sim=similarity(q, p)
            sims.append(sim)
        print(sims)
        h=max(sims)
        high=sims.index(h)
        print("high: ", high)
        for gt in egoschema_ans:
            if question_id==gt:
                print(gt)
                gt_num=egoschema_ans[gt]
                print("gt_num: ", gt_num)
                if high==gt_num:
                    if p_i==0:
                        ans1+=1
                    elif p_i==1:
                        ans2+=1
                    elif p_i==2:
                        ans3+=1

        p_i+=1
        # if p_i>0:
        #     break


    i+=1
    if i>10:
        break

promt1_acc=ans1/i
promt2_acc=ans2/i
promt3_acc=ans3/i
print("promt1_acc: ", promt1_acc)
print("promt2_acc: ", promt2_acc)
print("promt3_acc: ", promt3_acc)

# print(currentpred)
# print(i)
# accuracy=currentpred/i
# print(accuracy)

500
['0074f737-11cb-497d-8d07-77c3a8127391', '00b9a0de-c59e-49cb-a127-6081e2fb8c8e', '00f93e1e-cf4e-4835-88b4-4ad68216e86f', '00faf954-74f7-4aa3-8b29-4a5dff4f9518', '011b8b73-0ce4-4843-95ef-33b79610d212', '01a144a5-24d2-4a5a-af01-1f318d674bed', '026a2f15-c454-4c28-80e0-24c85d7f4ecf', '02925d7a-a5db-4127-8c31-b232e78b684d', '03657401-d4a4-40d0-9b03-d7e093ef93d1', '0437cf5f-5014-47d6-b4b3-f299380aa688', '049249dc-bdad-48c4-bdc0-511814c5781c', '04c51dba-1dcb-4b8f-a62c-efc363561d7b', '057f8774-15c2-4e2e-b9fd-75f26d4b3b83', '05ad5736-88f5-42bb-ac9f-689e199c50de', '05defeef-40bd-4b08-b341-72879a6cf63e', '05f1fc03-0c9e-4fd4-9d85-bb7be4e69234', '0688f66e-f115-49c6-85ff-712bf4f4a758', '06899b20-702f-450f-8422-2ae6dc9a6da8', '0725f410-3cf9-4f24-98ac-6610d1038ab1', '0750fa27-b049-4ded-9dd1-d495ffe23e75', '07a9d3ee-6d81-4bb4-ba24-f8555f8b1929', '07d7128e-0684-4c6b-81cb-3ecb76c2e6ec', '07fefc78-cfd2-4e04-8489-8ce4287158dd', '080eb552-9103-4f3e-a7e9-3417c1c0bcec', '083c5e8e-4546-40e9-857e-b9b967e500

In [25]:
import json
with open('esprompt_500.jsonl', 'r') as json_file1:
    pred = list(json_file1)
    pred_num=len(pred)
    print(pred_num)

with open('questions.json', 'r') as json_file2:
    questions = json.load(json_file2)
#     # qs_num=len(questions)
#     # print(qs_num)

with open('subset_answers.json', 'r') as json_file3:
    egoschema_ans = json.load(json_file3)
    gts=[]
    for gt in egoschema_ans:
        gts.append(gt)
    print(gts)
    gts_num=len(gts)
    print(gts_num)

ans1, ans2, ans3=0,0,0
i=0
# currentpred=0
for pred_list in pred:
    result = json.loads(pred_list)
    question_id=result['id']
    if question_id not in egoschema_ans:
        continue
    pred=result['pred']
    print("question_id: ", question_id)
    print("pred: ", pred)
    prompt=[]
    for ans in questions:
        if question_id==ans['q_uid']:
            prompt.append(ans['option 0'])
            prompt.append(ans['option 1'])
            prompt.append(ans['option 2'])
            prompt.append(ans['option 3'])
            prompt.append(ans['option 4'])
    print("prompt: ", prompt)
    sims=[]
    p_i=0
    for p in pred:
        # p_i+=1
        for q in prompt:
            sim=similarity(q, p)
            sims.append(sim)
        print(sims)
        h=max(sims)
        high=sims.index(h)
        print("high: ", high)
        for gt in egoschema_ans:
            if question_id==gt:
                print(gt)
                gt_num=egoschema_ans[gt]
                print("gt_num: ", gt_num)
                if high==gt_num:
                    if p_i==0:
                        ans1+=1
                    elif p_i==1:
                        ans2+=1
                    elif p_i==2:
                        ans3+=1

        p_i+=1
        # if p_i>0:
        #     break


    i+=1
    # if i>10:
    #     break

promt1_acc=ans1/i
promt2_acc=ans2/i
promt3_acc=ans3/i
print("promt1_acc: ", promt1_acc)
print("promt2_acc: ", promt2_acc)
print("promt3_acc: ", promt3_acc)

# print(currentpred)
# print(i)
# accuracy=currentpred/i
# print(accuracy)

500
['0074f737-11cb-497d-8d07-77c3a8127391', '00b9a0de-c59e-49cb-a127-6081e2fb8c8e', '00f93e1e-cf4e-4835-88b4-4ad68216e86f', '00faf954-74f7-4aa3-8b29-4a5dff4f9518', '011b8b73-0ce4-4843-95ef-33b79610d212', '01a144a5-24d2-4a5a-af01-1f318d674bed', '026a2f15-c454-4c28-80e0-24c85d7f4ecf', '02925d7a-a5db-4127-8c31-b232e78b684d', '03657401-d4a4-40d0-9b03-d7e093ef93d1', '0437cf5f-5014-47d6-b4b3-f299380aa688', '049249dc-bdad-48c4-bdc0-511814c5781c', '04c51dba-1dcb-4b8f-a62c-efc363561d7b', '057f8774-15c2-4e2e-b9fd-75f26d4b3b83', '05ad5736-88f5-42bb-ac9f-689e199c50de', '05defeef-40bd-4b08-b341-72879a6cf63e', '05f1fc03-0c9e-4fd4-9d85-bb7be4e69234', '0688f66e-f115-49c6-85ff-712bf4f4a758', '06899b20-702f-450f-8422-2ae6dc9a6da8', '0725f410-3cf9-4f24-98ac-6610d1038ab1', '0750fa27-b049-4ded-9dd1-d495ffe23e75', '07a9d3ee-6d81-4bb4-ba24-f8555f8b1929', '07d7128e-0684-4c6b-81cb-3ecb76c2e6ec', '07fefc78-cfd2-4e04-8489-8ce4287158dd', '080eb552-9103-4f3e-a7e9-3417c1c0bcec', '083c5e8e-4546-40e9-857e-b9b967e500